In [3]:
%cd /home/asehgal/neurosym-lib
from pathlib import Path
import numpy as np


/home/asehgal/neurosym-lib


/home/asehgal/neurosym-lib/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# Build standardized ECG datasets (single-label + multi-label).
raw_dir = Path("data/ecg_classification/ecg_process")
output_dir = Path("data/ecg_classification/ecg")
output_dir.mkdir(parents=True, exist_ok=True)

normalize_per_split = True
val_fraction = 0.15
seed = 42

x_train = np.load(raw_dir / "x_train.npy")
y_train = np.load(raw_dir / "y_train.npy")
x_test = np.load(raw_dir / "x_test.npy")
y_test = np.load(raw_dir / "y_test.npy")

if normalize_per_split:
    train_min = x_train.min(0)
    train_max = x_train.max(0)
    train_den = train_max - train_min
    train_den[train_den == 0] = 1.0
    x_train = (x_train - train_min) / train_den

    test_min = x_test.min(0)
    test_max = x_test.max(0)
    test_den = test_max - test_min
    test_den[test_den == 0] = 1.0
    x_test = (x_test - test_min) / test_den

rs = np.random.RandomState(seed)
indices = rs.permutation(len(x_train))
val_size = int(len(x_train) * val_fraction)
val_idx = indices[:val_size]
train_idx = indices[val_size:]

x_val = x_train[val_idx]
y_val = y_train[val_idx]
x_train = x_train[train_idx]
y_train = y_train[train_idx]

# Multi-label (multi-hot) labels
y_train_multi = y_train.astype(np.float32)
y_val_multi = y_val.astype(np.float32)
y_test_multi = y_test.astype(np.float32)

# Single-label labels (class indices) from argmax
y_train_single = y_train_multi.argmax(axis=-1).astype(np.int64).reshape(-1, 1)
y_val_single = y_val_multi.argmax(axis=-1).astype(np.int64).reshape(-1, 1)
y_test_single = y_test_multi.argmax(axis=-1).astype(np.int64).reshape(-1, 1)

np.savez_compressed(output_dir / "train_ecg_data.npz", x_train.astype(np.float32))
np.savez_compressed(output_dir / "valid_ecg_data.npz", x_val.astype(np.float32))
np.savez_compressed(output_dir / "test_ecg_data.npz", x_test.astype(np.float32))

np.savez_compressed(output_dir / "train_ecg_labels_multi.npz", y_train_multi)
np.savez_compressed(output_dir / "valid_ecg_labels_multi.npz", y_val_multi)
np.savez_compressed(output_dir / "test_ecg_labels_multi.npz", y_test_multi)

np.savez_compressed(output_dir / "train_ecg_labels_single.npz", y_train_single)
np.savez_compressed(output_dir / "valid_ecg_labels_single.npz", y_val_single)
np.savez_compressed(output_dir / "test_ecg_labels_single.npz", y_test_single)

print(f"train: {x_train.shape}, labels multi {y_train_multi.shape}, single {y_train_single.shape}")
print(f"val: {x_val.shape}, labels multi {y_val_multi.shape}, single {y_val_single.shape}")
print(f"test: {x_test.shape}, labels multi {y_test_multi.shape}, single {y_test_single.shape}")


train: (4092, 144), labels multi (4092, 9), single (4092, 1)
val: (721, 144), labels multi (721, 9), single (721, 1)
test: (2064, 144), labels multi (2064, 9), single (2064, 1)
